In [1]:
import networkx as nx
import os
import pandas as pd
import pickle

from datasketch import MinHash, MinHashLSH

#### Read LittleSis orgs

In [2]:
littlesis_orgs = {}

with open(os.getcwd() + '/Pickle/all_orgs.pickle', 'rb') as pkl:
    littlesis_orgs = pickle.load(pkl)
    
len(littlesis_orgs)

78526

#### Read bipartite graph

In [3]:
bipartite_all = nx.read_gpickle(os.getcwd() + '/Pickle/bipartite-all010420.pickle')
bipartite_all.number_of_nodes()

4054832

In [6]:
bipartite_orgs = list(bipartite_all.nodes(data = 'Uppername'))  # nodes which don't have this key return None eg. (bipartite_id, None)
bipartite_orgs[: 5]

[(0, None), (1, None), (2, None), (3, None), (4, None)]

In [7]:
bipartite_orgs = [(bipartite_id, uppername) for bipartite_id, uppername in bipartite_orgs if uppername is not None]  # remove nodes where uppername is None i.e. not organisations
bipartite_orgs[: 5]

[(8388614, 'MISSIONS UNLIMITED'),
 (8388623, 'CORVALLIS-OSU SYMPHONY SOCIETY'),
 (8388624, 'OLD CHURCH SOCIETY INC'),
 (8388646, 'TVW INC'),
 (8388664, 'EDWARD F ARNOLD TRUST')]

In [8]:
len(bipartite_orgs)

696376